In [1]:
import numpy as np
from datetime import datetime
from string import ascii_uppercase
import pandas as pd
import json
import psycopg2
import requests
from bs4 import BeautifulSoup
from psycopg2.extras import execute_values

In [2]:
def get_uuid(cur, cardName, cardSet, source):
    """
    This accesses the database to find the uuid for a card from a source, such as ck or csi
    :param cardName:
    :param cardSet:
    :param source: abbreviation for the data source: csi, ck etc.
    :return:
    """
    #To prevent a bug where single quotes in the name don't work
    if "'" in cardName:
        cardName = cardName.replace("'", "''")
    if "'" in cardSet:
        cardSet = cardSet.replace("'", "''")

    SQL = 'SELECT uuid FROM cardinfo WHERE ' + source +'name=\'' + cardName + '\' AND ' + source +'set=\'' + cardSet + '\';'
    
    cur.execute(SQL)
    query_results = cur.fetchall()
    
    return query_results[0][0]

In [3]:
def get_properties(cur, uuid, source):
    SQL = 'SELECT ' + source + 'Name, ' + source + 'Set FROM CardInfo WHERE uuid = \'' + uuid + '\';'
    cur.execute(SQL)
    query_results = cur.fetchall()

    return query_results[0]

In [4]:
def cardinfo_set_specific_generation(setcode, df, cards_df):
    for index, row in cards_df.iterrows():
        if setcode == '2XM':
            if type(row.borderColor) == str and row.borderColor == 'borderless':
                df['csSet'][index] = 'Double Masters Box Toppers'
                df['ckSet'][index] = 'Double Masters Box Toppers'
                df['csiSet'][index] = 'Double Masters: Variants'
                df.at[index, 'mmSet'] = 'Double Masters Box Topper'
        if setcode == 'M21':
            # Set the set names and CK names
            if type(row.frameEffects) == list and 'extendedart' in row.frameEffects:
                df['csSet'][index] = 'Core 2021 - Extended Art'
                df['ckSet'][index] = 'Core Set 2021 Variants'
                df['csiSet'][index] = 'Core Set 2021: Variants'
                df.at[index, 'mmSet'] = 'Core Set 2021 (Collector Edition)'
                df['ckName'][index] = df['name'][index] + ' (Extended Art)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Extended Art)'
            elif type(row.frameEffects) == list and 'showcase' in row.frameEffects:
                df['csSet'][index] = 'Core 2021 - Alternate Art'
                df['ckSet'][index] = 'Core Set 2021 Variants'
                df['csiSet'][index] = 'Core Set 2021: Variants'
                df.at[index, 'mmSet'] = 'Core Set 2021 (Collector Edition)'
                df['ckName'][index] = df['name'][index] + ' (Showcase)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Showcase)'
            elif type(row.borderColor) == str and row.borderColor == 'borderless':
                df['csSet'][index] = 'Core 2021 - Alternate Art'
                df['ckSet'][index] = 'Core Set 2021 Variants'
                df['csiSet'][index] = 'Core Set 2021: Variants'
                df.at[index, 'mmSet'] = 'Core Set 2021 (Collector Edition)'
                df['ckName'][index] = df['name'][index] + ' (Borderless)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Borderless)'
        elif setcode == 'ZNR':
            # Set the set names and CK names
            if type(row.frameEffects) == list and 'extendedart' in row.frameEffects:
                df['csSet'][index] = 'Zendikar Rising - Extended Art'
                df['ckSet'][index] = 'Zendikar Rising Variants'
                df['csiSet'][index] = 'Zendikar Rising: Variants'
                df.at[index, 'mmSet'] = 'Zendikar Rising (Collector Edition)'
                df['ckName'][index] = df['name'][index] + ' (Extended Art)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Extended Art)'
            elif type(row.frameEffects) == list and 'showcase' in row.frameEffects:
                df['csSet'][index] = 'Zendikar Rising - Showcase'
                df['ckSet'][index] = 'Zendikar Rising Variants'
                df['csiSet'][index] = 'Zendikar Rising: Variants'
                df.at[index, 'mmSet'] = 'Zendikar Rising (Collector Edition)'
                df['ckName'][index] = df['name'][index] + ' (Showcase)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Showcase)'
            elif type(row.borderColor) == str and row.borderColor == 'borderless':
                df['csSet'][index] = 'Zendikar Rising - Alternate Art'
                df['ckSet'][index] = 'Zendikar Rising Variants'
                df['csiSet'][index] = 'Zendikar Rising: Variants'
                df.at[index, 'mmSet'] = 'Zendikar Rising (Collector Edition)'
                df['ckName'][index] = df['name'][index] + ' (Borderless)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Borderless)'
        elif setcode == 'IKO':
            # Set the set names and CK names
            if type(row.frameEffects) == list and 'extendedart' in row.frameEffects:
                df['csSet'][index] = 'Ikoria: Lair of Behemoths - Extended Art'
                df['ckSet'][index] = 'Ikoria: Lair of Behemoths Variants'
                df['csiSet'][index] = 'Ikoria: Lair of Behemoths: Variants'
                df['ckName'][index] = df['name'][index] + ' (Extended Art)'
                df.at[index, 'mmSet'] = 'Ikoria: Lair of Behemoths (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Extended Art)'
            elif type(row.frameEffects) == list and 'showcase' in row.frameEffects:
                df['csSet'][index] = 'Ikoria: Lair of Behemoths - Alternate Art'
                df['ckSet'][index] = 'Ikoria: Lair of Behemoths Variants'
                df['csiSet'][index] = 'Ikoria: Lair of Behemoths: Variants'
                df['ckName'][index] = df['name'][index] + ' (Showcase)'
                df.at[index, 'mmSet'] = 'Ikoria: Lair of Behemoths (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Showcase)'
            elif type(row.flavorName) == str:
                df['csSet'][index] = 'Ikoria: Lair of Behemoths - Godzilla Series'
                df['ckSet'][index] = 'Ikoria: Lair of Behemoths Variants'
                df['csiSet'][index] = 'Ikoria: Lair of Behemoths: Variants'
                df['ckName'][index] = df['name'][index] + ' (' + row.flavorName + ' - Godzilla Series)'
                df.at[index, 'mmSet'] = 'Ikoria: Lair of Behemoths (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Godzilla Series)'
            elif type(row.borderColor) == str and row.borderColor == 'borderless':
                df['csSet'][index] = 'Ikoria: Lair of Behemoths - Alternate Art'
                df['ckSet'][index] = 'Ikoria: Lair of Behemoths Variants'
                df['csiSet'][index] = 'Ikoria: Lair of Behemoths: Variants'
                df['ckName'][index] = df['name'][index] + ' (Borderless)'
                df.at[index, 'mmSet'] = 'Ikoria: Lair of Behemoths (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Borderless)'
        elif setcode == 'C20':
            df['csiSet'][index] = 'Commander 2020 Edition'
            df.at[index, 'mmSet'] = 'Commander 2020: Ikoria'
        elif setcode == 'CMB1':
            df['csSet'][index] = 'Mystery Booster'
            df['ckSet'][index] = 'Mystery Booster/The List'
            df['ckName'][index] = df['name'][index] + ' (Not Tournament Legal)'
            df.at[index, 'mmSet'] = 'Mystery Booster'
        elif setcode == 'MB1':
            df.at[index, 'ckSet'] = 'Mystery Booster/The List'
        elif setcode == 'THB':
            # Set the set names and CK names
            if type(row.frameEffects) == list and 'extendedart' in row.frameEffects:
                df['csSet'][index] = 'Theros: Beyond Death - Extended Art'
                df['ckSet'][index] = 'Theros Beyond Death Variants'
                df['csiSet'][index] = 'Theros Beyond Death: Variants'
                df['ckName'][index] = df['name'][index] + ' (Extended Art)'
                df.at[index, 'mmSet'] = 'Theros Beyond Death (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Extended Art)'
            elif type(row.frameEffects) == list and 'showcase' in row.frameEffects:
                df['csSet'][index] = 'Theros: Beyond Death - Alternate Frame'
                df['ckSet'][index] = 'Theros Beyond Death Variants'
                df['csiSet'][index] = 'Theros Beyond Death: Variants'
                df['ckName'][index] = df['name'][index] + ' (Showcase)'
                df.at[index, 'mmSet'] = 'Theros Beyond Death (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Showcase)'
            elif type(row.borderColor) == str and row.borderColor == 'borderless':
                df['csSet'][index] = 'Theros: Beyond Death - Alternate Art'
                df['ckSet'][index] = 'Theros Beyond Death Variants'
                df['csiSet'][index] = 'Theros Beyond Death: Variants'
                df['ckName'][index] = df['name'][index] + ' (Borderless)'
                df.at[index, 'mmSet'] = 'Theros Beyond Death (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Borderless)'
            else:
                df['csSet'][index] = 'Theros: Beyond Death'
                df['csiSet'][index] = 'Theros Beyond Death'
        elif setcode == 'ELD':
            # Set the set names and CK names
            if type(row.frameEffects) == list and 'extendedart' in row.frameEffects:
                df['csSet'][index] = 'Throne of Eldraine - Extended Art'
                df['ckSet'][index] = 'Throne of Eldraine Variants'
                df['csiSet'][index] = 'Throne of Eldraine: Variants'
                df['ckName'][index] = df['name'][index] + ' (Extended Art)'
                df.at[index, 'mmSet'] = 'Throne of Eldraine (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Extended Art)'
            elif type(row.frameEffects) == list and 'showcase' in row.frameEffects:
                df['csSet'][index] = 'Throne of Eldraine - Alternate Frame'
                df['ckSet'][index] = 'Throne of Eldraine Variants'
                df['csiSet'][index] = 'Throne of Eldraine: Variants'
                df['ckName'][index] = df['name'][index] + ' (Showcase)'
                df.at[index, 'mmSet'] = 'Throne of Eldraine (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Showcase)'
            elif type(row.borderColor) == str and row.borderColor == 'borderless':
                df['csSet'][index] = 'Throne of Eldraine - Alternate Art'
                df['ckSet'][index] = 'Throne of Eldraine Variants'
                df['csiSet'][index] = 'Throne of Eldraine: Variants'
                df['ckName'][index] = df['name'][index] + ' (Borderless)'
                df.at[index, 'mmSet'] = 'Throne of Eldraine (Collector Edition)'
                df.at[index, 'mmName'] = df.at[index, 'name'] + ' (Borderless)'
        elif setcode == 'M20':
            df['csSet'][index] = 'Core 2020'
            df['csiSet'][index] = 'Core Set 2020'
        elif setcode == '10E':
            df.at[index, 'ckSet'] = '10th Edition'
            df.at[index, 'csSet'] = '10th Edition'
            df.at[index, 'mmSet'] = '10th Edition'
        elif setcode == '9ED':
            df.at[index, 'ckSet'] = '9th Edition'
            df.at[index, 'csSet'] = '9th Edition'
            df.at[index, 'mmSet'] = '9th Edition'
        elif setcode == '8ED':
            df.at[index, 'ckSet'] = '8th Edition'
            df.at[index, 'csSet'] = '8th Edition'
            df.at[index, 'mmSet'] = '8th Edition'
        elif setcode == '7ED':
            df.at[index, 'ckSet'] = '7th Edition'
            df.at[index, 'csSet'] = '7th Edition'
            df.at[index, 'mmSet'] = '7th Edition'
        elif setcode == '6ED':
            df.at[index, 'ckSet'] = '6th Edition'
            df.at[index, 'csSet'] = '6th Edition'
            df.at[index, 'mmSet'] = '6th Edition'
        elif setcode == '5ED':
            df.at[index, 'ckSet'] = '5th Edition'
            df.at[index, 'csSet'] = '5th Edition'
            df.at[index, 'mmSet'] = '5th Edition'
        elif setcode == '4ED':
            df.at[index, 'ckSet'] = '4th Edition'
            df.at[index, 'csSet'] = '4th Edition'
            df.at[index, 'mmSet'] = '4th Edition'
        elif setcode == 'CM2':
            df.at[index, 'mmSet'] = 'Commander Anthology Volume 2'
        # MM typo
        elif setcode == 'CN2':
            df.at[index, 'mmSet'] = 'Conpsiracy: Take the Crown'
        elif setcode == 'JOU':
            df.at[index, 'mmSet'] = 'Journey Into Nyx'
        elif setcode == 'MMA':
            df.at[index, 'mmSet'] = 'Modern Masters 2013'
        elif setcode == 'SOI':
            df.at[index, 'mmSet'] = 'Shadows Over Innistrad'

        # deal with split cards
        if '//' in df.at[index, 'name']:
            df.at[index, 'mmName'] = df.at[index, 'name'].replace('//', '/')

    # Loop over all duel decks
    codes = ['DD1', 'DD2']
    for letter in ascii_uppercase[2:-5]:
        code = 'DD' + letter
        codes.append(code)
    for c in codes:
        # MM typo
        if c == 'DDT':
            if setcode == c:
                for index, row in df.iterrows():
                    df.at[index, 'mmSet'] = 'Duel Decks: Merfolk vs Goblins'
        if setcode == c:
            for index, row in df.iterrows():
                nam = df.at[index, 'setName']
                adj = nam.replace('vs.', 'Vs.')
                df.at[index, 'ckSet'] = adj

                nam = df.at[index, 'setName']
                adj = nam.replace('Duel Decks', 'Duel Deck')
                df.at[index, 'csiSet'] = adj


    return df

In [5]:
def store_set(cards_object, setcode, setname):
    cards_df = pd.DataFrame(cards_object)

    try:
        tcg_ids = []
        for line in cards_df['identifiers'].values:
            try:
                tcg_ids.append(line['tcgplayerProductId'])
            except:
                print('A card in ' + setname + ' didn\'t work')
                tcg_ids.append(None)
        tcg_ids = pd.Series(tcg_ids)
        cards_df['tcgplayerProductId'] = tcg_ids

        standard_legal = []
        pioneer_legal = []
        modern_legal = []
        commander_legal = []
        for line in cards_df['legalities'].values:
            try:
                legalities = list(line.keys())
                standard_legal.append('standard' in legalities)
                pioneer_legal.append('pioneer' in legalities)
                modern_legal.append('modern' in legalities)
                commander_legal.append('commander' in legalities)
            except:
                print('A card in ' + setname + ' didn\'t work')
                tcg_ids.append(None)
        standard_legal = pd.Series(standard_legal)
        pioneer_legal = pd.Series(pioneer_legal)
        modern_legal = pd.Series(modern_legal)
        commander_legal = pd.Series(commander_legal)
        cards_df['standard'] = standard_legal
        cards_df['pioneer'] = pioneer_legal
        cards_df['modern'] = modern_legal
        cards_df['commander'] = commander_legal

        if setname in ['Ikoria: Lair of Behemoths']:
            table = cards_df[
                ['uuid', 'name', 'flavorName', 'frameEffects', 'borderColor', 'tcgplayerProductId', 'standard', 'pioneer', 'modern', 'commander']].copy()
        else:
            try:
                table = cards_df[['uuid', 'name', 'frameEffects', 'borderColor', 'tcgplayerProductId', 'standard', 'pioneer', 'modern', 'commander']].copy()
            except:
                table = cards_df[['uuid', 'name', 'borderColor', 'tcgplayerProductId', 'standard', 'pioneer', 'modern', 'commander']].copy()
                table['frameEffects'] = np.nan
            table['flavorName'] = np.nan
    except:
        print(setname + ' didn\'t work!')
        return
    table['setName'] = setname
    table['ckName'] = table['name']
    table['ckSet'] = table['setName']
    table['csiSet'] = table['setName']
    table['csiName'] = table['name']
    table['csSet'] = table['setName']
    table['csName'] = table['name']
    table['mmSet'] = table['setName']
    table['mmName'] = table['name']

    # pd.set_option('display.max_columns', None)
    # pd.set_option('display.max_rows', None)
    # print(cards_df.head(80))

    table = cardinfo_set_specific_generation(setcode, table, cards_df)

    table.drop_duplicates(subset=['tcgplayerProductId'], keep='first', ignore_index=True, inplace=True)
    
    conn = psycopg2.connect(host="localhost", port=5432, database="", user="", password="")
    cur = conn.cursor()
    
    t_df = tuple(table[['uuid', 'name', 'setName', 'flavorName', 'frameEffects', 'borderColor', 'ckName', 'ckSet', 'csiName', 'csiSet', 'csName', 'csSet', 'mmName', 'mmSet', 'tcgplayerProductId', 'standard', 'pioneer', 'modern', 'commander']].itertuples(index=False, name=None))
    execute_values(cur, "INSERT INTO cardinfo (uuid, cardname, setname, flavorname, frameeffects, bordercolor, ckname, ckset, csiname, csiset, csname, csset, mmname, mmset, tcgplayerproductid, standard, pioneer, modern, commander) VALUES %s " + 
                   """ON CONFLICT (uuid) DO UPDATE SET cardname = EXCLUDED.cardname, setname = EXCLUDED.setname, flavorname = EXCLUDED.flavorname, """ +
                   "frameeffects = EXCLUDED.frameeffects, bordercolor = EXCLUDED.bordercolor, ckname = EXCLUDED.ckname, " +
                   "ckset = EXCLUDED.ckset, csiname = EXCLUDED.csiname, csiset = EXCLUDED.csiset, csname = EXCLUDED.csname, " + 
                   "csset = EXCLUDED.csset, mmname = EXCLUDED.mmname, mmset = EXCLUDED.mmset, tcgplayerproductid = EXCLUDED.tcgplayerproductid, " +
                   "standard = EXCLUDED.standard, pioneer = EXCLUDED.pioneer, modern = EXCLUDED.modern, commander = EXCLUDED.commander;", t_df)

#     for index, row in table.iterrows():
#         try:
#             cur.execute("UPDATE cardinfo SET standard=%s, pioneer=%s, modern=%s, commander=%s WHERE uuid=%s", (row.standard, row.pioneer, row.modern, row.commander, row.uuid))
#         except Exception as e:
#             print('A card in ' + setname + ' didn\'t work with legality!')
    
    conn.commit()
    
    cur.close()
    conn.close()

#     for index, row in table.iterrows():
#         try:
#             with cnxn.cursor() as crsr:
#                 crsr.execute(
#                     "INSERT INTO CardInfo(uuid, cardName, setName, flavorName, frameEffects, borderColor, tcgplayerProductId, ckName, ckSet, csiName, csiSet, csName, csSet, mmName, mmSet) VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
#                     row['uuid'], row['name'], row.setName, row.flavorName, str(row.frameEffects), row.borderColor,
#                     row.tcgplayerProductId, row.ckName, row.ckSet, row.csiName, row.csiSet, row.csName, row.csSet,
#                     row.mmName, row.mmSet)
#         except Exception as e:
#             print('A card in ' + setname + ' didn\'t work!')

In [6]:
def pull_allprintings(store=True):
    data = requests.get('https://mtgjson.com/api/v5/AllPrintings.json').json()

    # Get the data date from the metadata
    # Metadata only contains this and the mtgjson version
    data_date = datetime.strptime(data['meta']['date'], '%Y-%m-%d')

    # Condense the rest of the data so that it's easier to refer to
    data = data['data']

    # A list of all set codes in the data file
    set_codes = data.keys()

    for set_code in set_codes:
        # Is it online only
        online_only = data[set_code]['isOnlineOnly']
        if online_only:
            continue

        # Number of cards in the set (Excluding promos etc)
        set_size = data[set_code]['baseSetSize']

        # Name of set
        set_name = data[set_code]['name']

        # # TCGPlayer Group ID
        # tcg_group_id = data[set_code]['tcgplayerGroupId']

        # Store the data in card_info
        if store:
            store_set(data[set_code]['cards'], set_code, set_name)

#         if get_edh_rank:
#             store_edhrec_rank(data[set_code]['cards'], set_name, data_date)

In [7]:
def main():
    pull_allprintings()

In [8]:
if __name__ == '__main__':
    main()

A card in Unlimited Edition didn't work
A card in Double Masters didn't work


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


A card in Revised Edition didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign Black Border didn't work
A card in Fourth Edition Foreign B

A card in Fifth Edition didn't work
A card in Fifth Edition didn't work
A card in Classic Sixth Edition didn't work
A card in Classic Sixth Edition didn't work
A card in Seventh Edition didn't work
Modern Horizons Art Series didn't work!
A card in Arabian Nights didn't work
A card in Arabian Nights didn't work
A card in Arabian Nights didn't work


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to

A card in Time Spiral Remastered didn't work
A card in Time Spiral Remastered didn't work
A card in Time Spiral Remastered didn't work
A card in From the Vault: Transform didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 2000 didn't work
A card in World Championship Decks 20

A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in World Championship Decks 2004 didn't work
A card in Wo

A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work
A card in Zendikar Rising didn't work


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to b